For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords

from collections import Counter

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import spacy
nlp = spacy.load('en')

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

## Load files

In [5]:
#filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
#          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']
filename='brady.txt'

In [6]:
file=open(filename,'r')
text=file.read()
file.close

<function TextIOWrapper.close()>

In [7]:
stops=set(stopwords.words("english"))
def text_cleaner(text):
    
    text=text.replace('gutenberg','')
    text=re.sub(r'ï»¿','',text)
    text=re.sub(r'\n','',text)
    text=re.sub('\[[^()]*\]','',text)
    text=re.sub('\([^()]*\)','',text)
    text=re.sub('[^\w\.\,]',' ',text)
    
    text=' '.join(text.split())
    text=text.split()
    text=[w for w in text if not w in stops]
    
    return text

In [8]:
text=text_cleaner(text)

In [9]:
text

['The',
 'Project',
 'Gutenberg',
 'eBook,',
 'And',
 'Thus',
 'He',
 'Came,',
 'Cyrus',
 'Townsend',
 'Brady,Illustrated',
 'Walter',
 'B.',
 'EverettThis',
 'eBook',
 'use',
 'anyone',
 'anywhere',
 'cost',
 'withalmost',
 'restrictions',
 'whatsoever.',
 'You',
 'may',
 'copy',
 'it,',
 'give',
 'away',
 'orre',
 'use',
 'terms',
 'Project',
 'Gutenberg',
 'License',
 'includedwith',
 'eBook',
 'online',
 'www..netTitle',
 'And',
 'Thus',
 'He',
 'CameAuthor',
 'Cyrus',
 'Townsend',
 'BradyRelease',
 'Date',
 'January',
 '5,',
 '2005',
 'Language',
 'EnglishCharacter',
 'set',
 'encoding',
 'ISO',
 '646',
 'US',
 'START',
 'OF',
 'THE',
 'PROJECT',
 'GUTENBERG',
 'EBOOK',
 'AND',
 'THUS',
 'HE',
 'CAME',
 'E',
 'text',
 'prepared',
 'Robert',
 'Cicconetti,',
 'Melissa',
 'Er',
 'Raqabi,',
 'ProjectGutenberg',
 'Online',
 'Distributed',
 'Proofreading',
 'Team',
 'Note',
 'Project',
 'Gutenberg',
 'also',
 'HTML',
 'version',
 'file',
 'includes',
 'original',
 'illustrations.',
 'Se

In [10]:
x

NameError: name 'x' is not defined

In [ ]:
def load_text(filename):
    file=open(filename,'r')
    text=file.read()
    file.close
    text=nlp(text)
    return text

In [ ]:
sentences=pd.DataFrame()
for i in range(0,len(filename)):
    text=load_text(filename[i])
    author=filename[i].replace('.txt','')
    text_sents=[[sent,author] for sent in text.sents]
    text_df=pd.DataFrame(text_sents)
    sentences=sentences.append(text_df)

In [ ]:
sentences

In [ ]:
# Utility function to create a list of the 10 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(10)]

In [ ]:
common_words=set()
for i in range(0,len(filename)):
    text=load_text(filename[i])
    words=bag_of_words(text)
    common_words.update(words)

In [ ]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(index=range(len(sentences)),columns=common_words)
    print(df)
    df['text_sentence'] = sentences[0].values
    print(df)
    df['text_source'] = sentences[1].values
    print(df)
    df.loc[:,common_words] = 0
    
    print()
    print(df)
    print(df.shape)
    
    print(df['text_source'])
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
       
    return df

In [ ]:
type(common_words)

In [ ]:
len(common_words)

In [ ]:
type(sentences)

In [ ]:
len(sentences)

In [ ]:
word_counts = bow_features(sentences, common_words)

In [ ]:
x

In [ ]:
stop_list=stopwords.words("english")

In [ ]:
type(stop_list)

In [ ]:
stop_list.sort()

In [ ]:
stop_list

In [ ]:
df.shape

In [ ]:
'[1075]' in common_words

In [ ]:
'[1075]' in sentences

In [ ]:
len

In [ ]:
def text_cleaner(text):
    text=text.replace('gutenberg','')
    text=re.sub(r'ï»¿','',text)
    text=re.sub(r'\n','',text)
    text=re.sub('\[[^()]*\]','',text)
    text=re.sub('\([^()]*\)','',text)
    text=re.sub('[^\w\.\,]',' ',text)
    
    text=' '.join(text.split())
    text=text.split()
    
    return text

In [ ]:
df = pd.DataFrame()

for i in range(len(filename)):
    text = text_creater(filename[i])
    text_clean = text_cleaner(text)
    
    text_doc=nlp(text_clean)
    text_words=bag_of_words(text_doc)
    
    

In [ ]:
df.shape

In [ ]:
from sklearn.preprocessing import normalize
x=df.drop(['author'],axis=1)
x=normalize(x)
y=df['author']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.25,
                                                    random_state=0)

In [ ]:
k=10
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(x_train)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
from sklearn import metrics
    
metrics.adjusted_rand_score(y_test, y_pred)

In [ ]:
y_pred=model.predict(x_train)
metrics.adjusted_rand_score(y_train, y_pred)

In [ ]:
from sklearn.metrics import pairwise_distances

model.fit(x)
labels=model.labels_

score=metrics.silhouette_score(x, labels, metric='euclidean')
print(score)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

lr = LogisticRegressionCV(cv=4)
lr.fit(x,y)
lr.score(x,y)

In [ ]:
from sklearn import ensemble
rfc = ensemble.RandomForestClassifier()
train = rfc.fit(x_train, y_train)

print('Training set score:', rfc.score(x_train, y_train))
print('\nTest set score:', rfc.score(x_test, y_test))